<a href="https://colab.research.google.com/github/NirantK/Hinglish/blob/mergeall/misc/MajorityVoting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
l = pd.read_csv("test_labels_hinglish.txt")
ref= pd.read_csv("/content/drive/My Drive/BERT_valid64.csv")
DistilBert1 = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/DistilBERT_lr_302e_05-test-full-output.csv")
DistilBert2 = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/DistilBERT_lr_513e_05-test-full-output.csv")
DistilBert3 = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/DistilBERT_lr_4e_05-test-full-output.csv")
Bert1 = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/BERT_lr_513e_05-test-full-output.csv")
Bert2 = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/BERT_lr_501e_05-test-full-output.csv")

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(ref["actual"])
clfs = [DistilBert1,DistilBert2, DistilBert3, Bert1, Bert2]
for df in clfs:
    df['y'] = le.transform(df["Sentiment"])


In [ ]:
classes_ = np.asarray([clf["y"] for clf in clfs])
maj = np.asarray([np.argmax(np.bincount(classes_[:,c])) for c in range(classes_.shape[1])])

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(precision_recall_fscore_support(maj, le.transform(ref["actual"]), average="macro"))
print(accuracy_score(maj, le.transform(ref["actual"])))

(0.6538055862391803, 0.6468791034770015, 0.647792003429256, None)
0.6458823529411765


In [ ]:
DistilBert1_final = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/DistilBERT_lr_302e_05-final_test-full-output.csv")
DistilBert2_final = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/DistilBERT_lr_513e_05-final_test-full-output.csv")
DistilBert3_final = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/DistilBERT_lr_4e_05-final_test-full-output.csv")
Bert1_final = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/BERT_lr_513e_05-final_test-full-output.csv")
Bert2_final = pd.read_csv("/content/drive/My Drive/HinglishNLP/PredOutput/BERT_lr_501e_05-final_test-full-output.csv")")

In [ ]:
clfs = [DistilBert1_final, DistilBert2_final, DistilBert3_final, Bert1_final, Bert2_final]
for df in clfs:
    df['y'] = le.transform(df["Sentiment"])

In [ ]:
classes_ = np.asarray([clf["y"] for clf in clfs])
maj = np.asarray([np.argmax(np.bincount(classes_[:,c])) for c in range(classes_.shape[1])])

In [ ]:
print(precision_recall_fscore_support(maj, le.transform(l["Sentiment"][:-1]), average="macro"))
print(accuracy_score(maj, le.transform(l["Sentiment"][:-1])))

(0.7098077999123605, 0.7071301529157158, 0.7076057816087483, None)
0.7042347449149716
